In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

/home/jyao01/.local/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
f = pd.read_csv("./data/result_face_url_2022_raw.csv")

In [4]:
f.shape

(1681, 5)

In [5]:
f.head()

,wmpid,face_url_2022,face_category_2022,face_2022,full_name
0,WMPID1299,https://s3.amazonaws.com/ballotpedia-api4/file...,Sen,yes,John Boozman
1,WMPID1302,https://s3.amazonaws.com/ballotpedia-api4/file...,Sen,yes,Mark Kelly
2,WMPID1305,https://s3.amazonaws.com/ballotpedia-api4/file...,Sen,yes,Michael Bennet
3,WMPID1315,https://s3.amazonaws.com/ballotpedia-api4/file...,Sen,yes,Richard Blumenthal
4,WMPID1321,https://s3.amazonaws.com/ballotpedia-api4/file...,Sen,yes,Marco Rubio


### Replace 'placeholder' and 'url expire' images from Ballotpedia with np.nan

In [6]:
f.loc[f['face_url_2022'].str.contains('placeholder', na=False, case=False)|f['face_url_2022'].str.contains('scontent', na=False, case=False)]

,wmpid,face_url_2022,face_category_2022,face_2022,full_name
23,WMPID1966,https://cdn.ballotpedia.org/images/f/fb/Silhou...,Sen,yes,Richard Adam Bowers
35,WMPID1859,https://cdn.ballotpedia.org/images/f/fb/Silhou...,Sen,yes,Aaron David Trevi?O
37,WMPID1773,https://cdn.ballotpedia.org/images/f/fb/Silhou...,Sen,yes,Teal'C Anderson
64,WMPID3496,https://cdn.ballotpedia.org/images/f/fb/Silhou...,Sen,yes,Jason Gabehart
65,WMPID241,https://cdn.ballotpedia.org/images/f/fb/Silhou...,Sen,yes,Ellen Kizik
...,...,...,...,...,...
1052,WMPID3315,https://cdn.ballotpedia.org/images/f/fb/Silhou...,House,yes,Jim Kidd Kidd
1053,WMPID3010,https://cdn.ballotpedia.org/images/f/fb/Silhou...,House,yes,George Kellogg
1164,WMPID4997,https://scontent-lga3-2.xx.fbcdn.net/v/t39.308...,House,yes,Guy Mclendon
1318,WMPID5152,https://cdn.ballotpedia.org/images/f/fb/Silhou...,House,yes,James Milo Howard


In [7]:
f.loc[f['face_url_2022'].str.contains('placeholder', na=False, case=False)|f['face_url_2022'].str.contains('scontent', na=False, case=False), 'face_url_2022'] =np.nan

In [8]:
# Total obs nan for face urls
f.face_url_2022.isna().sum()

97

### Search votesmart results for missing face urls

In [9]:
vs = pd.read_csv("./votesmart/vs_2022_face_cleaned.csv")

In [10]:
vs.shape

(1272, 5)

In [11]:
vs.head()

,vs_name,url,vs_state,vs_office,district_no
0,Patricia Chesbro,https://static.votesmart.org/static/canphoto/6...,Alaska,U.S. Senate,NaN
1,Ted Gianoutsos,https://static.votesmart.org/static/canphoto/2...,Alaska,U.S. Senate,NaN
2,Shoshana Gungerstein,https://static.votesmart.org/static/canphoto/2...,Alaska,U.S. Senate,NaN
3,Sid Hill,https://static.votesmart.org/static/canphoto/1...,Alaska,U.S. Senate,NaN
4,Buzz Kelley,https://static.votesmart.org/static/canphoto/2...,Alaska,U.S. Senate,NaN


In [12]:
f2 = f.loc[f.face_url_2022.isna()]

In [13]:
f2.face_category_2022.value_counts()

House    62
Sen      35
Name: face_category_2022, dtype: int64

In [14]:
f2.head()

,wmpid,face_url_2022,face_category_2022,face_2022,full_name
23,WMPID1966,NaN,Sen,yes,Richard Adam Bowers
35,WMPID1859,NaN,Sen,yes,Aaron David Trevi?O
37,WMPID1773,NaN,Sen,yes,Teal'C Anderson
46,WMPID1644,NaN,Sen,yes,Salomon Sr. Hernandez
49,WMPID3554,NaN,Sen,yes,Eric Chase Johnson


In [15]:
f2.shape

(97, 5)

In [16]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    """
    :param df_1: the left table to join
    :param df_2: the right table to join
    :param key1: key column of the left table
    :param key2: key column of the right table
    :param threshold: how close the matches should be to return a match, based on Levenshtein distance
    :param limit: the amount of matches that will get returned, these are sorted high to low
    :return: dataframe with boths keys and matches
    """
    s = df_2[key2].tolist()
    
    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2
    
    return df_1

In [17]:
dat = fuzzy_merge(f2, vs, 'full_name', 'vs_name', threshold=90)

/software/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/software/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [18]:
dat = dat.loc[dat['matches']!='']

In [19]:
dat.head()

,wmpid,face_url_2022,face_category_2022,face_2022,full_name,matches
65,WMPID241,NaN,Sen,yes,Ellen Kizik,Ellen Kizik
125,WMPID2092,NaN,Sen,yes,Walter Steve Sluzynsky,Walter Sluzynsky
138,WMPID4530,NaN,Sen,yes,Tommy Williams,Tommy Williams
141,WMPID4493,NaN,Sen,yes,C. Mark Coester,Mark Coester
247,WMPID5162,NaN,House,yes,Bill Powers,Bill Powers


In [20]:
dat =dat[['full_name', 'matches']]

In [21]:
dat.shape

(10, 2)

In [22]:
vs = vs.merge(dat, left_on='vs_name', right_on='matches', how='left')

In [23]:
vs.loc[vs.full_name.notna()]

,vs_name,url,vs_state,vs_office,district_no,full_name,matches
51,Roger Daugherty,https://static.votesmart.org/static/canphoto/2...,Arkansas,U.S. House,1,Roger Daugherty,Roger Daugherty
268,Nelson Rimensnyder,https://static.votesmart.org/static/canphoto/1...,District of Columbi,a U.S. Hous,Delegat,Nelson Rimensnyder,Nelson Rimensnyder
389,Judith Won Pat,https://static.votesmart.org/static/canphoto/2...,Guam,U.S. House,Delegate,Judith T Won Pat,Judith Won Pat
464,Ellen Kizik,https://static.votesmart.org/static/canphoto/2...,Indiana,U.S. Senate,NaN,Ellen Kizik,Ellen Kizik
473,Bill Powers,https://static.votesmart.org/static/canphoto/1...,Indiana,U.S. House,1,Bill Powers,Bill Powers
565,Scott Collier,https://static.votesmart.org/static/canphoto/1...,Maryland,U.S. House,7,Scott Collier,Scott Collier
996,Walter Sluzynsky,https://static.votesmart.org/static/canphoto/2...,Pennsylvania,U.S. House,17,Walter Steve Sluzynsky,Walter Sluzynsky
1162,Tommy Williams,https://static.votesmart.org/static/canphoto/1...,Utah,U.S. Senate,NaN,Tommy Williams,Tommy Williams
1203,Stacey Plaskett,https://static.votesmart.org/static/canphoto/1...,Virgin Islands,U.S. House,Delegate,Stacey Plaskett,Stacey Plaskett
1204,Mark Coester,https://static.votesmart.org/static/canphoto/2...,Vermont,U.S. Senate,NaN,C. Mark Coester,Mark Coester


Manually verify the above 10 face urls. Only the following five are legit

In [24]:
f.loc[f['full_name']=='Nelson Rimensnyder', 'face_url_2022'] = vs.loc[268, 'url']

In [25]:
f.loc[f['full_name']=='Ellen Kizik', 'face_url_2022'] = vs.loc[464, 'url']

In [26]:
f.loc[f['full_name']=='Scott Collier', 'face_url_2022'] = vs.loc[565, 'url']

In [27]:
#f.loc[f['full_name']=='Walter Steve Sluzynsky', 'face_url_2022']
f.loc[125, 'face_url_2022'] = vs.loc[996, 'url']

In [28]:
f.loc[f['full_name']=='Stacey Plaskett', 'face_url_2022'] = vs.loc[1203, 'url']

In [29]:
f.face_url_2022.isna().sum()

92

### Use ballotpedia's larger version of face images

In [30]:
f['face_url_2022'] = np.where(f['face_url_2022'].isna(), f['face_url_2022'],f['face_url_2022'].str.replace('/100/100/', '/200/300/'))

In [31]:
f = f.reset_index()

In [32]:
f.head()

,index,wmpid,face_url_2022,face_category_2022,face_2022,full_name
0,0,WMPID1299,https://s3.amazonaws.com/ballotpedia-api4/file...,Sen,yes,John Boozman
1,1,WMPID1302,https://s3.amazonaws.com/ballotpedia-api4/file...,Sen,yes,Mark Kelly
2,2,WMPID1305,https://s3.amazonaws.com/ballotpedia-api4/file...,Sen,yes,Michael Bennet
3,3,WMPID1315,https://s3.amazonaws.com/ballotpedia-api4/file...,Sen,yes,Richard Blumenthal
4,4,WMPID1321,https://s3.amazonaws.com/ballotpedia-api4/file...,Sen,yes,Marco Rubio


In [33]:
f['filename'] = f['wmpid']+'_'+ f['index'].astype(str) + '.jpg'

In [34]:
f['wget']='wget -O '+ f['filename'] + ' ' +f['face_url_2022']

In [35]:
f.head()

,index,wmpid,face_url_2022,face_category_2022,face_2022,full_name,filename,wget
0,0,WMPID1299,https://s3.amazonaws.com/ballotpedia-api4/file...,Sen,yes,John Boozman,WMPID1299_0.jpg,wget -O WMPID1299_0.jpg https://s3.amazonaws.c...
1,1,WMPID1302,https://s3.amazonaws.com/ballotpedia-api4/file...,Sen,yes,Mark Kelly,WMPID1302_1.jpg,wget -O WMPID1302_1.jpg https://s3.amazonaws.c...
2,2,WMPID1305,https://s3.amazonaws.com/ballotpedia-api4/file...,Sen,yes,Michael Bennet,WMPID1305_2.jpg,wget -O WMPID1305_2.jpg https://s3.amazonaws.c...
3,3,WMPID1315,https://s3.amazonaws.com/ballotpedia-api4/file...,Sen,yes,Richard Blumenthal,WMPID1315_3.jpg,wget -O WMPID1315_3.jpg https://s3.amazonaws.c...
4,4,WMPID1321,https://s3.amazonaws.com/ballotpedia-api4/file...,Sen,yes,Marco Rubio,WMPID1321_4.jpg,wget -O WMPID1321_4.jpg https://s3.amazonaws.c...


In [36]:
f.shape

(1681, 8)

In [37]:
f.face_url_2022.isna().sum()

92

In [38]:
with open('./data/face_urls.sh', 'w') as df:
  for i in f['wget']:
    df.write(str(i)+'\n')

In [39]:
mkdir ./face_img_raw

In [40]:
cd face_img_raw

/home/jyao01/github/face_url_scraper_2022/face_img_raw


In [41]:
!bash ../data/face_urls.sh

wget: /software/anaconda3/lib/libuuid.so.1: no version information available (required by wget)
--2023-03-25 11:09:59--  https://s3.amazonaws.com/ballotpedia-api4/files/thumbs/200/300/johnboozmanupdated.jpg
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.133.24, 52.217.142.40, 52.216.59.120, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.133.24|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10027 (9.8K) [image/jpeg]
Saving to: ‘WMPID1299_0.jpg’

100%[======================================>] 10,027      --.-K/s   in 0s      

2023-03-25 11:09:59 (88.0 MB/s) - ‘WMPID1299_0.jpg’ saved [10027/10027]

wget: /software/anaconda3/lib/libuuid.so.1: no version information available (required by wget)
--2023-03-25 11:09:59--  https://s3.amazonaws.com/ballotpedia-api4/files/thumbs/200/300/Mark-Kelly.png
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.37.128, 52.217.76.118, 54.231.128.128, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)

HTTP request sent, awaiting response... 200 OK
Length: 148988 (145K) [image/png]
Saving to: ‘WMPID4757_12.jpg’

100%[======================================>] 148,988     --.-K/s   in 0.03s   

2023-03-25 11:10:01 (4.64 MB/s) - ‘WMPID4757_12.jpg’ saved [148988/148988]

wget: /software/anaconda3/lib/libuuid.so.1: no version information available (required by wget)
--2023-03-25 11:10:01--  https://s3.amazonaws.com/ballotpedia-api4/files/thumbs/200/300/ShannonBray.jpg
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.48.62, 52.216.225.107, 52.217.226.152, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.48.62|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8563 (8.4K) [image/jpeg]
Saving to: ‘WMPID1396_13.jpg’

100%[======================================>] 8,563       --.-K/s   in 0s      

2023-03-25 11:10:01 (61.8 MB/s) - ‘WMPID1396_13.jpg’ saved [8563/8563]

wget: /software/anaconda3/lib/libuuid.so.1: no version information available (required

Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.226.152, 52.216.238.237, 52.217.116.32, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.226.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 126271 (123K) [image/png]
Saving to: ‘WMPID1969_26.jpg’

100%[======================================>] 126,271     --.-K/s   in 0.02s   

2023-03-25 11:10:03 (5.58 MB/s) - ‘WMPID1969_26.jpg’ saved [126271/126271]

wget: /software/anaconda3/lib/libuuid.so.1: no version information available (required by wget)
--2023-03-25 11:10:03--  https://s3.amazonaws.com/ballotpedia-api4/files/thumbs/200/300/Kenneth_Cates.jpg
Resolving s3.amazonaws.com (s3.amazonaws.com)... 54.231.202.192, 52.217.14.206, 54.231.129.72, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|54.231.202.192|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10257 (10K) [image/jpeg]
Saving to: ‘WMPID1592_27.jpg’

100%[======================================>] 10,25

Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.21.173|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13044 (13K) [image/jpeg]
Saving to: ‘WMPID3659_41.jpg’

100%[======================================>] 13,044      --.-K/s   in 0s      

2023-03-25 11:10:05 (96.6 MB/s) - ‘WMPID3659_41.jpg’ saved [13044/13044]

wget: /software/anaconda3/lib/libuuid.so.1: no version information available (required by wget)
--2023-03-25 11:10:05--  https://s3.amazonaws.com/ballotpedia-api4/files/thumbs/200/300/Dennis_Misigoy.jpg
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.137.248, 52.217.115.152, 52.217.9.166, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.137.248|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10168 (9.9K) [image/jpeg]
Saving to: ‘WMPID176_42.jpg’

100%[======================================>] 10,168      --.-K/s   in 0s      

2023-03-25 11:10:05 (93.9 MB/s) - ‘WMPID176_42.jpg’ saved [10168/10168]



HTTP request sent, awaiting response... 200 OK
Length: 7745 (7.6K) [image/jpeg]
Saving to: ‘WMPID2511_56.jpg’

100%[======================================>] 7,745       --.-K/s   in 0s      

2023-03-25 11:10:07 (135 MB/s) - ‘WMPID2511_56.jpg’ saved [7745/7745]

wget: /software/anaconda3/lib/libuuid.so.1: no version information available (required by wget)
--2023-03-25 11:10:07--  https://s3.amazonaws.com/ballotpedia-api4/files/thumbs/200/300/DavidRoth.jpg
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.137.248, 52.217.115.152, 52.217.9.166, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.137.248|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7501 (7.3K) [image/jpeg]
Saving to: ‘WMPID2576_57.jpg’

100%[======================================>] 7,501       --.-K/s   in 0s      

2023-03-25 11:10:07 (188 MB/s) - ‘WMPID2576_57.jpg’ saved [7501/7501]

wget: /software/anaconda3/lib/libuuid.so.1: no version information available (required by wge

...

Connecting to www.nga.org (www.nga.org)|35.222.161.56|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 100703 (98K) [image/jpeg]
Saving to: ‘WMPID5358_1631.jpg’

100%[======================================>] 100,703     --.-K/s   in 0.08s   

2023-03-25 11:13:36 (1.18 MB/s) - ‘WMPID5358_1631.jpg’ saved [100703/100703]

wget: /software/anaconda3/lib/libuuid.so.1: no version information available (required by wget)
--2023-03-25 11:13:36--  https://upload.wikimedia.org/wikipedia/commons/thumb/6/61/Anthony_Fauci_2020.jpg/220px-Anthony_Fauci_2020.jpg
Resolving upload.wikimedia.org (upload.wikimedia.org)... 208.80.154.240, 2620:0:861:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|208.80.154.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21563 (21K) [image/jpeg]
Saving to: ‘WMPID5215_1632.jpg’

100%[======================================>] 21,563      --.-K/s   in 0.007s  

2023-03-25 11:13:36 (2.78 MB/s) - ‘WMPID521

wget: /software/anaconda3/lib/libuuid.so.1: no version information available (required by wget)
--2023-03-25 11:13:40--  https://www.supremecourt.gov/about/justice_pictures/Kavanaugh
Resolving www.supremecourt.gov (www.supremecourt.gov)... 23.49.248.35, 23.49.248.6, 2600:141b:f000:d::684d:b2b8, ...
Connecting to www.supremecourt.gov (www.supremecourt.gov)|23.49.248.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 32263 (32K) [text/html]
Saving to: ‘WMPID16_1644.jpg’

100%[======================================>] 32,263      --.-K/s   in 0.006s  

2023-03-25 11:13:40 (5.01 MB/s) - ‘WMPID16_1644.jpg’ saved [32263/32263]

--2023-03-25 11:13:40--  http://12221_005_crop.jpg/
Resolving 12221_005_crop.jpg (12221_005_crop.jpg)... failed: Name or service not known.
wget: unable to resolve host address ‘12221_005_crop.jpg’
FINISHED --2023-03-25 11:13:40--
Total wall clock time: 0.1s
Downloaded: 1 files, 32K in 0.006s (5.01 MB/s)
wget: /software/anaconda3/lib/libuuid.s

HTTP request sent, awaiting response... 200 OK
Length: 14839 (14K) [image/jpeg]
Saving to: ‘WMPID1_1655.jpg’

100%[======================================>] 14,839      --.-K/s   in 0s      

2023-03-25 11:13:41 (141 MB/s) - ‘WMPID1_1655.jpg’ saved [14839/14839]

wget: /software/anaconda3/lib/libuuid.so.1: no version information available (required by wget)
--2023-03-25 11:13:41--  https://upload.wikimedia.org/wikipedia/commons/thumb/e/e1/Director_Robert_S._Mueller-_III.jpg/220px-Director_Robert_S._Mueller-_III.jpg
Resolving upload.wikimedia.org (upload.wikimedia.org)... 208.80.154.240, 2620:0:861:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|208.80.154.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16485 (16K) [image/jpeg]
Saving to: ‘WMPID8_1656.jpg’

100%[======================================>] 16,485      --.-K/s   in 0s      

2023-03-25 11:13:41 (106 MB/s) - ‘WMPID8_1656.jpg’ saved [16485/16485]

wget: /software/anaconda3/lib/l

HTTP request sent, awaiting response... 200 OK
Length: 12544 (12K) [image/jpeg]
Saving to: ‘WMPID5169_1667.jpg’

100%[======================================>] 12,544      --.-K/s   in 0s      

2023-03-25 11:13:42 (257 MB/s) - ‘WMPID5169_1667.jpg’ saved [12544/12544]

wget: /software/anaconda3/lib/libuuid.so.1: no version information available (required by wget)
--2023-03-25 11:13:42--  https://upload.wikimedia.org/wikipedia/commons/thumb/5/5f/LEN%C3%8DN_MORENO_SE_RE%C3%9ANE_CON_EL_L%C3%8DDER_MEXICANO_L%C3%93PEZ_OBRADOR_%2836186836092%29_%28cropped%29.jpg/220px-LEN%C3%8DN_MORENO_SE_RE%C3%9ANE_CON_EL_L%C3%8DDER_MEXICANO_L%C3%93PEZ_OBRADOR_%2836186836092%29_%28cropped%29.jpg
Resolving upload.wikimedia.org (upload.wikimedia.org)... 208.80.154.240, 2620:0:861:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|208.80.154.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15485 (15K) [image/jpeg]
Saving to: ‘WMPID5177_1668.jpg’

100%[==============

HTTP request sent, awaiting response... 200 OK
Length: 21872 (21K) [image/jpeg]
Saving to: ‘WMPID5248_1678.jpg’

100%[======================================>] 21,872      --.-K/s   in 0.007s  

2023-03-25 11:13:43 (2.78 MB/s) - ‘WMPID5248_1678.jpg’ saved [21872/21872]

wget: /software/anaconda3/lib/libuuid.so.1: no version information available (required by wget)
--2023-03-25 11:13:43--  https://upload.wikimedia.org/wikipedia/commons/thumb/0/0b/Mukhisa_Kituyi%2C_Houlin_Zhao%2C_Tedros_Adhanom_Ghebreyesus_with_Sophia_-_AI_for_Good_Global_Summit_2018_%2841223188035%29_%28cropped%29.jpg/220px-Mukhisa_Kituyi%2C_Houlin_Zhao%2C_Tedros_Adhanom_Ghebreyesus_with_Sophia_-_AI_for_Good_Global_Summit_2018_%2841223188035%29_%28cropped%29.jpg
Resolving upload.wikimedia.org (upload.wikimedia.org)... 208.80.154.240, 2620:0:861:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|208.80.154.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19762 (19K) [image/jpe

In [42]:
path = './'
filename=[]
filesize=[]
for i in os.listdir('.'):
    filename.append(i)
    size = os.path.getsize(os.path.join(path,i))
    filesize.append(size)

In [43]:
img = pd.DataFrame(columns=['filename', 'filesize'])

img['filename'] = filename
img['filesize'] = filesize

In [44]:
img.loc[img.filesize==0].shape

(51, 2)

In [45]:
d = f.merge(img, on='filename', how='left', indicator=True)

In [46]:
d.head()

,index,wmpid,face_url_2022,face_category_2022,face_2022,full_name,filename,wget,filesize,_merge
0,0,WMPID1299,https://s3.amazonaws.com/ballotpedia-api4/file...,Sen,yes,John Boozman,WMPID1299_0.jpg,wget -O WMPID1299_0.jpg https://s3.amazonaws.c...,10027.0,both
1,1,WMPID1302,https://s3.amazonaws.com/ballotpedia-api4/file...,Sen,yes,Mark Kelly,WMPID1302_1.jpg,wget -O WMPID1302_1.jpg https://s3.amazonaws.c...,140095.0,both
2,2,WMPID1305,https://s3.amazonaws.com/ballotpedia-api4/file...,Sen,yes,Michael Bennet,WMPID1305_2.jpg,wget -O WMPID1305_2.jpg https://s3.amazonaws.c...,8978.0,both
3,3,WMPID1315,https://s3.amazonaws.com/ballotpedia-api4/file...,Sen,yes,Richard Blumenthal,WMPID1315_3.jpg,wget -O WMPID1315_3.jpg https://s3.amazonaws.c...,10724.0,both
4,4,WMPID1321,https://s3.amazonaws.com/ballotpedia-api4/file...,Sen,yes,Marco Rubio,WMPID1321_4.jpg,wget -O WMPID1321_4.jpg https://s3.amazonaws.c...,10242.0,both


In [47]:
d._merge.value_counts()

both          1589
left_only       92
right_only       0
Name: _merge, dtype: int64

In [48]:
d.shape

(1681, 10)

In [49]:
len(d.wmpid.unique())

1650

### Manually go through all the images to make sure faces are not:
- severely cropped
- occluded 
- non-frontal

In [50]:
# fix Brett Kavanaugh image
d.loc[d['wmpid']=='WMPID16', 'face_url_2022'] = 'https://upload.wikimedia.org/wikipedia/commons/thumb/3/37/Associate_Justice_Brett_Kavanaugh_Official_Portrait_%28full_length%29.jpg/220px-Associate_Justice_Brett_Kavanaugh_Official_Portrait_%28full_length%29.jpg'

In [51]:
# Lacy Watson (severe shadow on face)
d.loc[d['wmpid']=='WMPID764', 'face_url_2022'] = 'https://bloximages.chicago2.vip.townnews.com/wvnews.com/content/tncms/assets/v3/editorial/d/79/d79ef528-35ca-5bd4-9ac7-0530b376f139/5ecde2049ae37.image.jpg?resize=334%2C500'

In [52]:
# Shante Munns (non-frontal face)
d.loc[d['wmpid']=='WMPID1572', 'face_url_2022'] = 'https://pbs.twimg.com/media/FoOlGA8XEAIx7Xr?format=jpg&name=large'

In [53]:
# Kat Cammack (non-frontal face)
d.loc[d['wmpid']=='WMPID124', 'face_url_2022'] = 'http://cammack.house.gov/sites/evo-subsites/cammack.house.gov/files/evo-media-image/rep-cammack-official.JPG'

In [54]:
# Shane Hazel (non-frontal face)
d.loc[d['wmpid']=='WMPID1332', 'face_url_2022'] = 'https://forsythnews.cdn-anvilcms.net/media/images/2018/03/28/images/Hazel_2.max-1200x675.jpg'

In [55]:
# Penny Ronning (non-frontal face)
d.loc[d['wmpid']=='WMPID1677', 'face_url_2022']='https://apps.montanafreepress.org/election-guide-2022/static/b7be0357c4fa2714cc24df8c19395b51/d6138/penny-ronning.png'

In [56]:
# fix Frank Bertone
d.loc[d['wmpid']=='WMPID2195', 'face_url_2022']='https://apps.arizona.vote/electioninfo/assets/32/0/CandidatePhotos/bertone-frank-19161.jpg'

In [57]:
# Mitchel Davil (face covered)
d.loc[d['wmpid']=='WMPID2211', 'face_url_2022']='https://media.licdn.com/dms/image/C4E03AQGFZzdPgUvBJw/profile-displayphoto-shrink_800_800/0/1663694851976?e=2147483647&v=beta&t=JABPCpk0mGzwVYydrmHBjHwM-lxd5HJccX51x9gnvTk'

In [58]:
# Madison Gesiotto Gilbert (non frontal face)
d.loc[d['wmpid']=='WMPID3527', 'face_url_2022']='https://pbs.twimg.com/profile_images/1536866017368068101/gArKTcOU_400x400.jpg'

In [59]:
# Allen Cannon (non frontal face)
d.loc[d['wmpid']=='WMPID5114', 'face_url_2022']='https://pbs.twimg.com/profile_images/559921560656416768/YHVTjpHt_400x400.jpeg'

In [60]:
# Kyrsten Sinema (non frontal face)
d.loc[d['wmpid']=='WMPID1526', 'face_url_2022']='https://pbs.twimg.com/profile_images/913536000579825671/fCG9ySli_400x400.jpg'

In [61]:
# Willie Montague (non frontal face)
d.loc[d['wmpid']=='WMPID2107', 'face_url_2022'] ='https://static.wixstatic.com/media/934aa2_7f29064785fe434d88366bead0d3234c~mv2.jpg/v1/fill/w_1623,h_858,al_c/934aa2_7f29064785fe434d88366bead0d3234c~mv2.jpg'

In [62]:
# Kirsten Engel (Cropped face)
d.loc[d['wmpid']=='WMPID2788', 'face_url_2022'] = 'https://www.gannett-cdn.com/presto/2021/03/26/PPHX/8d2ea18e-0f77-461c-b235-f2a67dcbaede-Engel.JPG?crop=1969,2712,x202,y220'

In [63]:
# Miranda Tozier-Robbins (Cropped face)
d.loc[d['wmpid']=='WMPID4480', 'face_url_2022'] ='https://pbs.twimg.com/profile_images/1472593683132428294/q43_pYTr_400x400.jpg'

In [64]:
d.loc[d['wmpid']=='WMPID4511', 'face_url_2022'] ='https://upload.wikimedia.org/wikipedia/commons/thumb/a/a0/Herschel_Walker_in_May_2018_%281%29.jpg/220px-Herschel_Walker_in_May_2018_%281%29.jpg'

In [65]:
# Robbie_Hoffman (occluded face)
d.loc[d['wmpid']=='WMPID5121', 'face_url_2022']= 'https://s3.amazonaws.com/ballotpedia-api4/files/thumbs/100/100/Robbie_Hoffman.JPG'

In [66]:
# Paul Cook (side face)
d.loc[d['wmpid']=='WMPID5156', 'face_url_2022'] ='https://s3.amazonaws.com/ballotpedia-api4/files/thumbs/200/300/Paul_Cook.jpg'

In [67]:
# Justin Trudeau (non frontal face)
d.loc[d['wmpid']=='WMPID5261', 'face_url_2022'] ='https://pm.gc.ca/themes/pm_theme/images/Trudeau-head.jpg'

In [68]:
# Fix Hank Schroeder
d.loc[d['wmpid']=='WMPID5123', 'face_url_2022'] = 'https://s3.amazonaws.com/ballotpedia-api4/files/thumbs/200/300/Hank_Schroeder.jpg'

In [69]:
# Mariannette Jane Miller-Meeks (cropped face)
d.loc[d['wmpid']=='WMPID2556', 'face_url_2022']='https://upload.wikimedia.org/wikipedia/commons/5/54/Mariannette_Miller-Meeks_117th_U.S_Congress.jpg'

In [70]:
# David Biery (artistic background)
d.loc[d['wmpid']=='WMPID2695', 'face_url_2022']='https://static.wixstatic.com/media/7d2849_1f733d59c4d84b1287d29f126979761d~mv2.jpg/v1/fill/w_715,h_1074,al_c,q_85,usm_0.66_1.00_0.01,enc_auto/7d2849_1f733d59c4d84b1287d29f126979761d~mv2.jpg'

### Get one final face image for each wmpid

In [71]:
# Use the largest img for each wmpid
d = d.sort_values(by=['wmpid', 'filesize'])

In [72]:
uni = d.drop_duplicates(subset=['wmpid'], keep='last')

In [73]:
uni.shape

(1650, 10)

In [74]:
uni.loc[uni['wget'].notna()].shape

(1558, 10)

In [75]:
uni.loc[uni['wget'].isna()].shape

(92, 10)

In [76]:
uni['wget2']='wget -O '+ uni['wmpid'] + '.jpg ' +uni['face_url_2022']

/software/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [77]:
uni.head(2)

,index,wmpid,face_url_2022,face_category_2022,face_2022,full_name,filename,wget,filesize,_merge,wget2
1655,1655,WMPID1,https://upload.wikimedia.org/wikipedia/commons...,PromPoli,yes,Mike Pence,WMPID1_1655.jpg,wget -O WMPID1_1655.jpg https://upload.wikimed...,14839.0,both,wget -O WMPID1.jpg https://upload.wikimedia.or...
1638,1638,WMPID10,https://www.supremecourt.gov/about/justice_pic...,SupremeCourt,yes,John Roberts,WMPID10_1638.jpg,wget -O WMPID10_1638.jpg https://www.supremeco...,9959.0,both,wget -O WMPID10.jpg https://www.supremecourt.g...


In [78]:
cd ..

/home/jyao01/github/face_url_scraper_2022


In [79]:
with open('./data/face_urls_uni.sh', 'w') as df:
  for i in uni['wget2']:
    df.write(str(i)+'\n')

In [80]:
mkdir ./face_collection_2022

In [81]:
cd face_collection_2022

/home/jyao01/github/face_url_scraper_2022/face_collection_2022


In [93]:
# Rerun this cell multiple times to avoid 'Too many requests' for wikipedia 
!bash ../data/face_urls_uni.sh

wget: /software/anaconda3/lib/libuuid.so.1: no version information available (required by wget)
--2023-03-25 11:35:53--  https://upload.wikimedia.org/wikipedia/commons/thumb/b/b9/Mike_Pence_official_Vice_Presidential_portrait.jpg/220px-Mike_Pence_official_Vice_Presidential_portrait.jpg
Resolving upload.wikimedia.org (upload.wikimedia.org)... 208.80.154.240, 2620:0:861:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|208.80.154.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14839 (14K) [image/jpeg]
Saving to: ‘WMPID1.jpg’

100%[======================================>] 14,839      --.-K/s   in 0s      

2023-03-25 11:35:53 (185 MB/s) - ‘WMPID1.jpg’ saved [14839/14839]

wget: /software/anaconda3/lib/libuuid.so.1: no version information available (required by wget)
--2023-03-25 11:35:53--  https://www.supremecourt.gov/about/justice_pictures/Roberts_8807-16_Crop.jpg
Resolving www.supremecourt.gov (www.supremecourt.gov)... 23.49.248.35, 23.49

Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.114.93|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9799 (9.6K) [image/jpeg]
Saving to: ‘WMPID1018.jpg’

100%[======================================>] 9,799       --.-K/s   in 0s      

2023-03-25 11:35:54 (101 MB/s) - ‘WMPID1018.jpg’ saved [9799/9799]

wget: /software/anaconda3/lib/libuuid.so.1: no version information available (required by wget)
--2023-03-25 11:35:54--  https://s3.amazonaws.com/ballotpedia-api4/files/thumbs/200/300/Brian_Babin_official_congressional_photo.jpg
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.169.208, 54.231.234.248, 54.231.235.0, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.169.208|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10914 (11K) [image/jpeg]
Saving to: ‘WMPID1020.jpg’

100%[======================================>] 10,914      --.-K/s   in 0s      

2023-03-25 11:35:54 (82.4 MB/s) - ‘WMPID1020.jpg’ saved [10

Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.125.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8465 (8.3K) [image/jpeg]
Saving to: ‘WMPID1041.jpg’

100%[======================================>] 8,465       --.-K/s   in 0s      

2023-03-25 11:35:56 (89.5 MB/s) - ‘WMPID1041.jpg’ saved [8465/8465]

wget: /software/anaconda3/lib/libuuid.so.1: no version information available (required by wget)
--2023-03-25 11:35:56--  https://s3.amazonaws.com/ballotpedia-api4/files/thumbs/200/300/460px-Debbie_Lesko.jpg
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.35.160, 52.217.125.112, 52.216.113.109, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.35.160|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12645 (12K) [image/jpeg]
Saving to: ‘WMPID1047.jpg’

100%[======================================>] 12,645      --.-K/s   in 0s      

2023-03-25 11:35:56 (134 MB/s) - ‘WMPID1047.jpg’ saved [12645/12645]

wget: /sof

HTTP request sent, awaiting response... 200 OK
Length: 11447 (11K) [image/jpeg]
Saving to: ‘WMPID1069.jpg’

100%[======================================>] 11,447      --.-K/s   in 0s      

2023-03-25 11:35:58 (112 MB/s) - ‘WMPID1069.jpg’ saved [11447/11447]

wget: /software/anaconda3/lib/libuuid.so.1: no version information available (required by wget)
--2023-03-25 11:35:58--  https://s3.amazonaws.com/ballotpedia-api4/files/thumbs/200/300/John-Briscoe.PNG
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.89.102, 52.217.45.126, 52.216.142.150, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.89.102|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 116185 (113K) [image/png]
Saving to: ‘WMPID1070.jpg’

100%[======================================>] 116,185     --.-K/s   in 0.02s   

2023-03-25 11:35:58 (5.15 MB/s) - ‘WMPID1070.jpg’ saved [116185/116185]

wget: /software/anaconda3/lib/libuuid.so.1: no version information available (required by wget)

HTTP request sent, awaiting response... 200 OK
Length: 12974 (13K) [image/jpeg]
Saving to: ‘WMPID1091.jpg’

100%[======================================>] 12,974      --.-K/s   in 0s      

2023-03-25 11:36:00 (125 MB/s) - ‘WMPID1091.jpg’ saved [12974/12974]

wget: /software/anaconda3/lib/libuuid.so.1: no version information available (required by wget)
--2023-03-25 11:36:00--  https://s3.amazonaws.com/ballotpedia-api4/files/thumbs/200/300/_Maria_Elvira_Salazar_.jpg
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.89.102, 52.217.45.126, 52.216.142.150, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.89.102|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11679 (11K) [image/jpeg]
Saving to: ‘WMPID1093.jpg’

100%[======================================>] 11,679      --.-K/s   in 0s      

2023-03-25 11:36:00 (76.7 MB/s) - ‘WMPID1093.jpg’ saved [11679/11679]

wget: /software/anaconda3/lib/libuuid.so.1: no version information available (required b

...

Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.13.222|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11265 (11K) [image/jpeg]
Saving to: ‘WMPID910.jpg’

100%[======================================>] 11,265      --.-K/s   in 0s      

2023-03-25 11:39:07 (100 MB/s) - ‘WMPID910.jpg’ saved [11265/11265]

wget: /software/anaconda3/lib/libuuid.so.1: no version information available (required by wget)
--2023-03-25 11:39:07--  https://s3.amazonaws.com/ballotpedia-api4/files/thumbs/200/300/YvetteClarkeNY.png
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.109.254, 52.217.122.224, 52.217.13.222, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.109.254|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 155028 (151K) [image/png]
Saving to: ‘WMPID915.jpg’

100%[======================================>] 155,028     --.-K/s   in 0.08s   

2023-03-25 11:39:07 (1.97 MB/s) - ‘WMPID915.jpg’ saved [155028/155028]

wget: /softw

HTTP request sent, awaiting response... 200 OK
Length: 10118 (9.9K) [image/jpeg]
Saving to: ‘WMPID936.jpg’

100%[======================================>] 10,118      --.-K/s   in 0s      

2023-03-25 11:39:09 (114 MB/s) - ‘WMPID936.jpg’ saved [10118/10118]

wget: /software/anaconda3/lib/libuuid.so.1: no version information available (required by wget)
--2023-03-25 11:39:09--  https://s3.amazonaws.com/ballotpedia-api4/files/thumbs/200/300/Cathy_McMorris_Rodgers.jpg
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.13.222, 52.217.1.254, 52.216.62.104, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.13.222|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10548 (10K) [image/jpeg]
Saving to: ‘WMPID937.jpg’

100%[======================================>] 10,548      --.-K/s   in 0s      

2023-03-25 11:39:09 (66.7 MB/s) - ‘WMPID937.jpg’ saved [10548/10548]

wget: /software/anaconda3/lib/libuuid.so.1: no version information available (required by wge

Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.244.238, 54.231.203.120, 52.216.138.109, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.244.238|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10213 (10.0K) [image/jpeg]
Saving to: ‘WMPID952.jpg’

100%[======================================>] 10,213      --.-K/s   in 0.001s  

2023-03-25 11:39:11 (18.7 MB/s) - ‘WMPID952.jpg’ saved [10213/10213]

wget: /software/anaconda3/lib/libuuid.so.1: no version information available (required by wget)
--2023-03-25 11:39:11--  https://s3.amazonaws.com/ballotpedia-api4/files/thumbs/200/300/Nanette_Barragan.jpg
Resolving s3.amazonaws.com (s3.amazonaws.com)... 54.231.203.120, 52.216.138.109, 54.231.200.0, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|54.231.203.120|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10590 (10K) [image/jpeg]
Saving to: ‘WMPID954.jpg’

100%[======================================>] 10,590      --

HTTP request sent, awaiting response... 200 OK
Length: 10573 (10K) [image/jpeg]
Saving to: ‘WMPID968.jpg’

100%[======================================>] 10,573      --.-K/s   in 0.001s  

2023-03-25 11:39:12 (17.5 MB/s) - ‘WMPID968.jpg’ saved [10573/10573]

wget: /software/anaconda3/lib/libuuid.so.1: no version information available (required by wget)
--2023-03-25 11:39:12--  https://s3.amazonaws.com/ballotpedia-api4/files/thumbs/200/300/Kimberly_Walker.jpg
Resolving s3.amazonaws.com (s3.amazonaws.com)... 54.231.200.0, 52.216.52.224, 52.216.177.45, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|54.231.200.0|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8547 (8.3K) [image/jpeg]
Saving to: ‘WMPID969.jpg’

100%[======================================>] 8,547       --.-K/s   in 0s      

2023-03-25 11:39:12 (85.4 MB/s) - ‘WMPID969.jpg’ saved [8547/8547]

wget: /software/anaconda3/lib/libuuid.so.1: no version information available (required by wget)
--2023-

HTTP request sent, awaiting response... 200 OK
Length: 14556 (14K) [image/jpeg]
Saving to: ‘WMPID984.jpg’

100%[======================================>] 14,556      --.-K/s   in 0s      

2023-03-25 11:39:14 (94.3 MB/s) - ‘WMPID984.jpg’ saved [14556/14556]

wget: /software/anaconda3/lib/libuuid.so.1: no version information available (required by wget)
--2023-03-25 11:39:14--  https://s3.amazonaws.com/ballotpedia-api4/files/thumbs/200/300/Jamie_Raskin.jpg
Resolving s3.amazonaws.com (s3.amazonaws.com)... 54.231.203.120, 52.216.138.109, 54.231.200.0, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|54.231.203.120|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12003 (12K) [image/jpeg]
Saving to: ‘WMPID986.jpg’

100%[======================================>] 12,003      --.-K/s   in 0s      

2023-03-25 11:39:14 (264 MB/s) - ‘WMPID986.jpg’ saved [12003/12003]

wget: /software/anaconda3/lib/libuuid.so.1: no version information available (required by wget)
--202

In [94]:
path = './'
filename=[]
filesize=[]
for i in os.listdir('.'):
    filename.append(i)
    size = os.path.getsize(os.path.join(path,i))
    filesize.append(size)

In [95]:
img = pd.DataFrame(columns=['filename', 'filesize'])

img['filename'] = filename
img['filesize'] = filesize

img.loc[img.filesize==0].shape

(2, 2)

In [96]:
img.loc[img.filesize==0]

,filename,filesize
438,WMPID2195.jpg,0
442,WMPID2211.jpg,0


Manually check these two errors

In [97]:
uni.loc[uni['wmpid']=='WMPID2195', 'wget2'][383] # url forbidden; no good alternative

'wget -O WMPID2195.jpg https://apps.arizona.vote/electioninfo/assets/32/0/CandidatePhotos/bertone-frank-19161.jpg'

In [98]:
uni.loc[uni['wmpid']=='WMPID2211', 'wget2'][610] # fail to download; no good alternative

'wget -O WMPID2211.jpg https://media.licdn.com/dms/image/C4E03AQGFZzdPgUvBJw/profile-displayphoto-shrink_800_800/0/1663694851976?e=2147483647&v=beta&t=JABPCpk0mGzwVYydrmHBjHwM-lxd5HJccX51x9gnvTk'

remove the face images with 0 bytes

In [99]:
!rm WMPID2195.jpg WMPID2211.jpg

In [100]:
uni.loc[uni['wmpid']=='WMPID2195', 'face_url_2022']=np.nan
uni.loc[uni['wmpid']=='WMPID2211', 'face_url_2022']=np.nan

/software/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


### Create the master file for face urls

In [101]:
uni.head(2)

,index,wmpid,face_url_2022,face_category_2022,face_2022,full_name,filename,wget,filesize,_merge,wget2
1655,1655,WMPID1,https://upload.wikimedia.org/wikipedia/commons...,PromPoli,yes,Mike Pence,WMPID1_1655.jpg,wget -O WMPID1_1655.jpg https://upload.wikimed...,14839.0,both,wget -O WMPID1.jpg https://upload.wikimedia.or...
1638,1638,WMPID10,https://www.supremecourt.gov/about/justice_pic...,SupremeCourt,yes,John Roberts,WMPID10_1638.jpg,wget -O WMPID10_1638.jpg https://www.supremeco...,9959.0,both,wget -O WMPID10.jpg https://www.supremecourt.g...


In [102]:
uni = uni[['wmpid', 'full_name', 'face_url_2022', 'face_category_2022']]

In [104]:
uni.face_url_2022.isna().sum()

94

In [105]:
uni.head(2)

,wmpid,full_name,face_url_2022,face_category_2022
1655,WMPID1,Mike Pence,https://upload.wikimedia.org/wikipedia/commons...,PromPoli
1638,WMPID10,John Roberts,https://www.supremecourt.gov/about/justice_pic...,SupremeCourt


In [106]:
uni.to_csv("../result_face_url_2022.csv", index=False)